In [10]:
import pandas as pd
import numpy as np

In [3]:
user_events = pd.read_csv('Encrypted Datasets/filtered_user_events.csv')
user_segments = pd.read_csv('Encrypted Datasets/user_segment_slice.csv')
body_users = pd.read_csv('Encrypted Datasets/body_users.csv')

In [5]:
user_events.head()

,id,user_id,name,date_mod
0,778739492,5970559,Trade result successful,2025-01-01 00:00:08.018
1,778739498,8835246,User logged in,2025-01-01 00:00:09.433
2,778739499,8835246,Authentication completed,2025-01-01 00:00:09.440
3,778739501,8835246,User logged in,2025-01-01 00:00:10.109
4,778739502,8835246,Authentication completed,2025-01-01 00:00:10.114


In [7]:
# Преобразуем столбцы с датами в datetime
user_events['date_mod'] = pd.to_datetime(user_events['date_mod'])


In [11]:
# Анализ общей активности
user_activity = user_events.groupby('user_id').agg(
    total_events=('id', 'count'),
    unique_events=('name', 'nunique'),
    first_event_date=('date_mod', 'min'),
    last_event_date=('date_mod', 'max'),
    active_days=('date_mod', lambda x: x.dt.date.nunique())
).reset_index()

# Расчет дополнительных метрик
user_activity['activity_duration_days'] = (user_activity['last_event_date'] - user_activity['first_event_date']).dt.days
user_activity['events_per_day'] = user_activity['total_events'] / user_activity['activity_duration_days'].replace(0, 1)
user_activity['avg_daily_events'] = user_activity['total_events'] / user_activity['active_days'].replace(0, 1)

# Заменяем бесконечные значения на 0
user_activity = user_activity.replace([np.inf, -np.inf], 0)

In [12]:
user_activity.head()

,user_id,total_events,unique_events,first_event_date,last_event_date,active_days,activity_duration_days,events_per_day,avg_daily_events
0,151692,123,8,2025-01-15 16:20:32.654,2025-06-28 14:43:16.679,88,163,0.754601,1.397727
1,314546,5,4,2025-01-18 05:25:00.496,2025-05-19 15:05:26.468,3,121,0.041322,1.666667
2,399125,1,1,2025-01-18 04:16:48.089,2025-01-18 04:16:48.089,1,0,1.000000,1.000000
3,413506,1,1,2025-01-18 04:46:54.763,2025-01-18 04:46:54.763,1,0,1.000000,1.000000
4,775310,78,7,2025-01-18 08:26:07.463,2025-06-27 16:05:55.341,29,160,0.487500,2.689655


In [13]:
user_activity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419215 entries, 0 to 419214
Data columns (total 9 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   user_id                 419215 non-null  int64         
 1   total_events            419215 non-null  int64         
 2   unique_events           419215 non-null  int64         
 3   first_event_date        419215 non-null  datetime64[ns]
 4   last_event_date         419215 non-null  datetime64[ns]
 5   active_days             419215 non-null  int64         
 6   activity_duration_days  419215 non-null  int64         
 7   events_per_day          419215 non-null  float64       
 8   avg_daily_events        419215 non-null  float64       
dtypes: datetime64[ns](2), float64(2), int64(5)
memory usage: 28.8 MB
